In [ ]:
!pip3 install -U gensim

In [ ]:
import os
import sys
# import logging
import gensim
import pandas as pd
from gensim.models import Doc2Vec
# curPath = os.path.abspath(os.path.dirname(__file__))
# rootPath = os.path.split(curPath)[0]
# sys.path.append(rootPath)
# from utilties import ko_title2words

In [ ]:
# import gensim.downloader
# # load pre-trained model
# model = gensim.downloader.load('word2vec-google-news-300')

In [ ]:
main_data = pd.read_csv('news/TSLA_20201112.csv')
main_data.head()

In [ ]:
article_content = main_data['content']
article_content

In [ ]:
title_content = '. '.join([main_data['title'][0], main_data['content'][0]])
print(title_content)

In [ ]:
import nltk
# nltk.download('stopwords')

from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim import models
from gensim.models.doc2vec import TaggedDocument
# Function for normalizing paragraphs.
def normalize(string):
    lst = word_tokenize(string)
    lst = [word.lower() for word in lst if word.isalpha()]
    lst = [w for w in lst if not w in stopwords.words('english')]
    return lst
# Aggregate questions under each topic tag as a paragraph. 
# Normalize the paragraph 
# Feed the normalized paragraph along with the topic tag into Gensim's Tagged Document function. 
# Append the return value to docs.
title_content = normalize(title_content)
docs = list()
for i, doc in enumerate(title_content):
    # print(doc)
    docs.append(TaggedDocument(doc, [i]))

docs


In [ ]:

import gensim
model = gensim.models.Doc2Vec(vector_size=128, window=3, min_count=0, workers=4, epochs=40)
model.build_vocab(docs)
model.train(docs, total_examples=model.corpus_count, epochs=model.iter)
model.save("d2v.model")
print(models)

In [ ]:
import os
import re
import numpy
import pickle
import gensim
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from gensim.models import Doc2Vec

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

title_content = '. '.join([main_data['title'][0], main_data['content'][0]])
print(title_content)

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate([title_content])]
print(documents)
model = Doc2Vec(documents, vector_size=128, window=4, min_count=0, workers=4)
# print(model.docvecs)
inferred = model.infer_vector(["system", "oopo"])
sims = model.docvecs.most_similar([inferred])
print(sims)

In [ ]:
# 加载数据
documents = []
# 使用count当做每个句子的“标签”，标签和每个句子是一一对应的
count = 0
with open('../data/titles/ko.video.corpus','r') as f:
    for line in f:
        title = unicode(line, 'utf-8')
        # 切词，返回的结果是列表类型
        words = ko_title2words(title)
        # 这里documents里的每个元素是二元组，具体可以查看函数文档
        documents.append(gensim.models.doc2vec.TaggedDocument(words, [str(count)]))
        # count += 1
        # if count % 10000 == 0:
        #     logging.info('{} has loaded...'.format(count))

if not os.path.exists('models'):
    os.makedirs('models')

model = Doc2Vec(documents, dm=1, size=128, window=8, min_count=5)
model.save('models/eng_d2v.model')